#The Bert model fine tuning code is adapted from the huggingface and beautiful article and video shared by 
https://mccormickml.com/2019/07/22/BERT-fine-tuning/


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# # Any results you write to the current directory are saved as output.

In [ ]:

ann_df=pd.read_csv("/kaggle/input/10k-german-news-articles/Annotations.csv")
posts_df=pd.read_csv("/kaggle/input/10k-german-news-articles/Posts.csv")
cats_df=pd.read_csv("/kaggle/input/10k-german-news-articles/Categories.csv")
News_df=pd.read_csv("/kaggle/input/10k-german-news-articles/Newspaper_Staff.csv")
articles_df=pd.read_csv("/kaggle/input/10k-german-news-articles/Articles.csv")
cvsplit_df=pd.read_csv("/kaggle/input/10k-german-news-articles/CrossValSplit.csv")
consol_df=pd.read_csv("/kaggle/input/10k-german-news-articles/Annotations_consolidated.csv")


In [ ]:
ann_df.head()

In [ ]:
posts_df.head()

In [ ]:
cats_df.head()

In [ ]:
News_df.head()

In [ ]:
articles_df.head()

In [ ]:
cvsplit_df.head()

In [ ]:
consol_df.head()

In [ ]:
ann_df['Category'].unique()

In [ ]:
ann_df.info()

In [ ]:
articles_df.info()

In [ ]:
articles_df["Body"][0]

In [ ]:
posts_df["Body"][2]

In [ ]:
posts_df.info()

In [ ]:
consol_df.info()

In [ ]:
posts_df[posts_df["ID_Post"]==3326]

In [ ]:
result_Df = pd.merge(posts_df, consol_df, on="ID_Post")

In [ ]:
print(result_Df.info())


In [ ]:
result_Df.head()

In [ ]:
tr_Df = pd.merge(result_Df, articles_df, on="ID_Article")

In [ ]:
tr_Df.info()

In [ ]:
tr_Df.head(2)

In [ ]:
tr_Df['Body_y'][0]

In [ ]:
tr_Df["Category"].unique()

In [ ]:
tr_Df["Value"].unique()

I'm assuming the x values are "Body_y" column and The predictors as "Value" column

In [ ]:
import re
def clean_text(sent):
  
  sentence = sent.lower()
  S=re.sub('"','',sentence)
  S = re.sub(r'[^\w]', ' ', S)
  S = re.sub("\s\s+", " ", S)

  return S
tr_Df['Body_y']=tr_Df['Body_y'].apply(clean_text)

In [ ]:
tr_Df['Body_y'][0:3]

In [ ]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, y_train, y_test = train_test_split( tr_Df['Body_y'], tr_Df['Value'], test_size=0.20, random_state=42,stratify=tr_Df['Category'])

In [ ]:
train=pd.DataFrame(pd.concat([X_train,y_train],axis=1))
test=pd.DataFrame(pd.concat([X_test,y_test],axis=1))

In [ ]:
train.head()

In [ ]:
!pip install transformers

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [ ]:
def gen_id_mask(data):
  sms = data.Body_y.values

  # We need to add special tokens at the beginning and end of each sentence for BERT to work properly
  sms = ["[CLS] " + sm + " [SEP]" for sm in sms]
  labels = data.Value.values

  tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-german-cased')

  tokenized_texts = [tokenizer.tokenize(sm) for sm in sms]
  

  MAX_LEN = 256
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  attention_masks = []

  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

  print('success')
  return (input_ids,labels,attention_masks)

In [ ]:
train_input_ids,train_labels,train_input_masks=gen_id_mask(train)
test_input_ids,test_labels,test_input_masks=gen_id_mask(test)

In [ ]:
train_input_ids[0]

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if torch.cuda.is_available():    

    # use the GPU.    
    device = torch.device("cuda")
    print('GPU Device name is :', torch.cuda.get_device_name(0))
model_bert = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-german-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model_bert.cuda()

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#Converting the inputs for model into torch tensors
train_inputs = torch.tensor(train_input_ids)
validation_inputs = torch.tensor(test_input_ids)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(test_labels)
train_masks = torch.tensor(train_input_masks)
validation_masks = torch.tensor(test_input_masks)



# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from tqdm import tqdm, trange

import io


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import random
from transformers import get_linear_schedule_with_warmup



#Code is adapted from following path (run_glue.py)
#https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128



# Prepare optimizer and schedule (linear warmup and decay)
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#         {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#         {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#         ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=2e-3)
param_optimizer = list(model_bert.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


optimizer = AdamW(optimizer_grouped_parameters,lr=1e-5)
# Number of training epochs (authors recommend between 2 and 4)


# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * 10 #The value 10 is no of epochs

# Create the learning rate scheduler.
# scheduler = get_linear_schedule_with_warmup(optimizer, 
#                                             num_warmup_steps = 0, # Default value in run_glue.py
#                                             num_training_steps = total_steps)

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss and accuracies after each epoch so we can plot them.
loss_values = []
Train_Master_accuracy=[]
Validation_Master_accuracy=[]
epochs=2
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model_bert.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 50 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model_bert.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model_bert(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model_bert.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        #scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ====================================================
    #               Validation and Training Accuracies
    # ====================================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model_bert.eval()

    # Tracking variables 
    train_loss, train_accuracy = 0, 0
    nb_train_steps, nb_train_examples = 0, 0

    # Evaluate data for one epoch
    for batch in train_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model_bert(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_train_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        train_accuracy += tmp_train_accuracy

        # Track the number of batches
        nb_train_steps += 1

    # Report the final accuracy for this validation run.
    print("  Train Accuracy: {0:.2f}".format(train_accuracy/nb_train_steps))
    Train_Master_accuracy.append(train_accuracy/nb_train_steps)
    print("  Accuracy evaluation on training took: {:}".format(format_time(time.time() - t0)))

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model_bert(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Validation Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    Validation_Master_accuracy.append(eval_accuracy/nb_eval_steps)
    print("  Validation took: {:}".format(format_time(time.time() - t0)))